In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr = LinearRegression(featuresCol="features",labelCol="label",
                      predictionCol="prediction")

In [8]:
lrModel=lr.fit(training)

In [9]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrModel.intercept

0.14228558260358093

In [11]:
training_summary = lrModel.summary


In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [14]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [15]:
train_data, test_data= all_data.randomSplit([0.7,0.3])

In [16]:
 train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 343|
|   mean|-0.06769135877397238|
| stddev|   10.45777787607665|
|    min| -28.571478869743427|
|    max|  27.111027963108548|
+-------+--------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                158|
|   mean| 0.9615154175896335|
| stddev| 10.003793045003052|
|    min|-22.949825936196074|
|    max|  27.78383192005107|
+-------+-------------------+



In [18]:
correct_model=lr.fit(train_data)

In [19]:
test_results = correct_model.evaluate(test_data)

In [20]:
test_results.rootMeanSquaredError

10.211540724572288

In [21]:
unlabeld_data = test_data.select("features")

In [22]:
unlabeld_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [25]:
predictions = correct_model.transform(unlabeld_data)

In [26]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  3.1637650114660776|
|(10,[0,1,2,3,4,5,...| -1.6308638359254284|
|(10,[0,1,2,3,4,5,...|  0.7792458494649965|
|(10,[0,1,2,3,4,5,...|  1.0920744073215654|
|(10,[0,1,2,3,4,5,...|  2.8650746837447993|
|(10,[0,1,2,3,4,5,...|   1.954973194648321|
|(10,[0,1,2,3,4,5,...|  0.8141731157012432|
|(10,[0,1,2,3,4,5,...| 0.40312222873828385|
|(10,[0,1,2,3,4,5,...| -1.9688411273270228|
|(10,[0,1,2,3,4,5,...| -0.1565496744452912|
|(10,[0,1,2,3,4,5,...|   2.321633824967503|
|(10,[0,1,2,3,4,5,...|-0.15550820323753342|
|(10,[0,1,2,3,4,5,...| -1.7507275370577422|
|(10,[0,1,2,3,4,5,...| -0.5687181651999841|
|(10,[0,1,2,3,4,5,...|  1.6824126134877713|
|(10,[0,1,2,3,4,5,...| -3.4968082293165472|
|(10,[0,1,2,3,4,5,...|  3.3436160650096447|
|(10,[0,1,2,3,4,5,...|  -4.617422938219769|
|(10,[0,1,2,3,4,5,...|  -2.500718649805526|
|(10,[0,1,2,3,4,5,...|   1.23176

In [27]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

In [29]:
from pyspark.ml.regression import LinearRegression

In [30]:
df = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,
                   header=True)

In [31]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [32]:
df.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [35]:
for item in df.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [36]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [37]:
df.columns


['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [40]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'], outputCol="features")

In [41]:
output= assembler.transform(df)


In [43]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [46]:
final_data = output.select("features",'Yearly Amount Spent')

In [47]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [48]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [49]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean|  502.5676455384045|
| stddev|   79.9276264844712|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [50]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean| 492.20583891479293|
| stddev|  77.73728333633096|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [51]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [52]:
lr_model = lr.fit(train_data)

In [53]:
test_results = lr_model.evaluate(test_data)

In [55]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-1.28466126625426...|
|5.172751116333529...|
|-2.57500687439460...|
|-3.97903932025656...|
|1.119815351557917...|
|2.671640686457976...|
|6.536993168992922...|
|1.847411112976260...|
|2.501110429875552...|
|-2.84217094304040...|
|8.640199666842818...|
|4.433786671143025...|
|-6.42330633127130...|
|-7.84439180279150...|
|-2.25099938688799...|
|-4.32009983342140...|
|3.410605131648481...|
|1.307398633798584...|
|-2.50111042987555...|
|1.944044925039634...|
+--------------------+
only showing top 20 rows



C:\Users\BDG-DAROBAYO\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [56]:
test_results.rootMeanSquaredError

1.0818271804008897e-11

In [57]:
test_results.r2


1.0

In [58]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [59]:
unlabeled_data = test_data.select("features")

In [60]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[31.0472221394875...|
|[31.0613251567161...|
|[31.5147378578019...|
|[31.5741380228732...|
|[31.6005122003032...|
|[31.6253601348306...|
|[31.7242025238451...|
|[31.8124825597242...|
|[31.8293464559211...|
|[31.8530748017465...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9365486184489...|
|[31.9563005605233...|
|[31.9673209478824...|
|[32.0123007682454...|
|[32.0215955013870...|
|[32.0444861274404...|
|[32.0478009788678...|
|[32.0609143984100...|
+--------------------+
only showing top 20 rows



In [63]:
predictions =lr_model.transform(unlabeled_data)

In [64]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.0472221394875...|392.49739918903424|
|[31.0613251567161...|487.55545805789643|
|[31.5147378578019...|489.81248799648716|
|[31.5741380228732...| 544.4092721605908|
|[31.6005122003032...|479.17285149108574|
|[31.6253601348306...| 376.3369007569215|
|[31.7242025238451...|503.38788728795396|
|[31.8124825597242...|392.81034498377875|
|[31.8293464559211...| 385.1523379879725|
|[31.8530748017465...| 459.2851234623548|
|[31.9120759292006...| 387.5347163056991|
|[31.9262720263601...|392.20493344432197|
|[31.9365486184489...| 427.1993848953346|
|[31.9563005605233...| 547.1259317472067|
|[31.9673209478824...|445.74984123967477|
|[32.0123007682454...|492.94505306596255|
|[32.0215955013870...|  521.572174757824|
|[32.0444861274404...| 448.2298291865369|
|[32.0478009788678...|  513.450571186099|
|[32.0609143984100...| 627.6033187129956|
+--------------------+------------